This notebook is done following 
* [Building text classifier with Differential Privacy](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)
* [Fine-tuning with custom datasets](https://huggingface.co/transformers/v3.4.0/custom_datasets.html#seq-imdb)

# Initial Setup
https://huggingface.co/docs/transformers/training

## Install

In [1]:
!pip install datasets
import datasets

## Import

In [2]:
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
import torch
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gc

pd.set_option('display.max_columns', None)

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tokenize-jigsaw-comments-using-bert/train_undersampled.csv
/kaggle/input/tokenize-jigsaw-comments-using-bert/__results__.html
/kaggle/input/tokenize-jigsaw-comments-using-bert/validation.pkl
/kaggle/input/tokenize-jigsaw-comments-using-bert/train.pkl
/kaggle/input/tokenize-jigsaw-comments-using-bert/test.pkl
/kaggle/input/tokenize-jigsaw-comments-using-bert/validation.csv
/kaggle/input/tokenize-jigsaw-comments-using-bert/__notebook__.ipynb
/kaggle/input/tokenize-jigsaw-comments-using-bert/__output__.json
/kaggle/input/tokenize-jigsaw-comments-using-bert/test.csv
/kaggle/input/tokenize-jigsaw-comments-using-bert/custom.css


## Config

In [4]:
from dataclasses import dataclass

@dataclass
class Config:
    model_name = 'bert-base-uncased'
    dataset_name = 'jigsaw-unintended-bias-in-toxicity-classification'
    text_column = 'comment_text'
    # if id column is string, replace that with an integer index during preprocessing
    id_column = 'id'

    # target in raw dataset. However, it will be renamed to `labels` here to facilitate training setup
    raw_target_column = 'toxicity'
    target_column = 'labels'
    
    undersample = True
    need_to_split = True
    
    test_size = 0.2
    max_seq_length = 128
    seed = 2022
    
    # train config
    batch_size = 64
    learning_rate = 1e-3
    epochs = 3
    num_labels = 2

## Set seed

In [5]:
import random

def seed_torch(seed=7):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

global_seed = Config.seed
seed_torch(global_seed)

## Get device

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


# Load tokenized data

From my [other notebook](https://www.kaggle.com/code/khairulislam/tokenize-jigsaw-comments). The dataset is tokenized from the [Jigsaw competition]( https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification) and [all_data.csv](https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification/data?select=all_data.csv)

In [7]:
text = Config.text_column
target = Config.target_column
root = '/kaggle/input/tokenize-jigsaw-comments-using-bert/'

In [8]:
import pickle
    
with open(root + 'train.pkl', 'rb') as input_file:
    train_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'validation.pkl', 'rb') as input_file:
    validation_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'test.pkl', 'rb') as input_file:
    test_tokenized = pickle.load(input_file)
    input_file.close()

In [9]:
print(train_tokenized)

Dataset({
    features: ['id', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 230934
})


In [10]:
id_column = Config.id_column # the original id column HITId has been replaced with index because it was string and torch didn't support str format
train_ids = train_tokenized[id_column]
test_ids = test_tokenized[id_column]
validation_ids = validation_tokenized[id_column]

train_tokenized = train_tokenized.remove_columns(id_column)
test_tokenized = test_tokenized.remove_columns(id_column)
validation_tokenized = validation_tokenized.remove_columns(id_column)

# Training

## Data loader

In [11]:
BATCH_SIZE = Config.batch_size

train_dataloader = DataLoader(train_tokenized, batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(validation_tokenized, batch_size=BATCH_SIZE*5)
test_dataloader = DataLoader(test_tokenized, batch_size=BATCH_SIZE*5)

## Model, hyper-parameters and callbacks

In [12]:
# add the utility script from File->Add utility script
from train_utils import TrainUtil, ModelCheckPoint, EarlyStopping

num_labels = Config.num_labels
model_name = Config.model_name
train_util = TrainUtil(Config.id_column, Config.target_column, device)

In [13]:
model = TrainUtil.load_pretrained_model(model_name, num_labels)

# Define optimizer
LEARNING_RATE = Config.learning_rate
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
EPOCHS = Config.epochs

# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True) 

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Total parameters count: 109483778
Trainable parameters count: 7680002, percent 7.015


In [14]:
result_dir = ''
best_model_path = os.path.join(result_dir, 'model.pt')

if result_dir != '':
    os.makedirs(result_dir, exist_ok=True)

check_point = ModelCheckPoint(filepath=best_model_path)
early_stopping = EarlyStopping(patience=3, min_delta=0)

## Loop

In [15]:
start_epoch = 1
# load a previous model if there is any
# model, optimizer, lr_scheduler, start_epoch = load_model(model, optimizer, lr_scheduler, device, filepath=best_model_path)
model = model.to(device)

for epoch in range(start_epoch, EPOCHS+1):
    gc.collect()
    
    train_loss, train_result, train_probs = train_util.train(model, train_dataloader, optimizer, epoch)
    val_loss, val_result, val_probs = train_util.evaluate(model, validation_dataloader, epoch, 'Validation')

    print(
      f"Epoch: {epoch} | "
      f"Train loss: {train_loss:.3f} | "
      f"Train result: {train_result} |\n"
      f"Validation loss: {val_loss:.3f} | "
      f"Validation result: {val_result} | "
    )
    
    loss = -val_result['f1']
    lr_scheduler.step(loss)
    check_point(model, optimizer, lr_scheduler, epoch, loss)
    
    early_stopping(loss)
    if early_stopping.early_stop:
        break
    print()

Epoch 1 (Train):   0%|          | 0/3609 [00:00<?, ?it/s]

Epoch 1 (Validation):   0%|          | 0/1129 [00:00<?, ?it/s]

Epoch: 1 | Train loss: 0.359 | Train result: {'accuracy': 0.8427, 'f1': 0.8429, 'auc': 0.9199} |
Validation loss: 0.350 | Validation result: {'accuracy': 0.8367, 'f1': 0.465, 'auc': 0.936} | 

Loss improved from inf to -0.465. Saving model.



Epoch 2 (Train):   0%|          | 0/3609 [00:00<?, ?it/s]

Epoch 2 (Validation):   0%|          | 0/1129 [00:00<?, ?it/s]

Epoch: 2 | Train loss: 0.340 | Train result: {'accuracy': 0.8527, 'f1': 0.8529, 'auc': 0.9287} |
Validation loss: 0.310 | Validation result: {'accuracy': 0.8617, 'f1': 0.4998, 'auc': 0.9383} | 

Loss improved from -0.465 to -0.500. Saving model.



Epoch 3 (Train):   0%|          | 0/3609 [00:00<?, ?it/s]

Epoch 3 (Validation):   0%|          | 0/1129 [00:00<?, ?it/s]

Epoch: 3 | Train loss: 0.333 | Train result: {'accuracy': 0.8558, 'f1': 0.8564, 'auc': 0.9317} |
Validation loss: 0.338 | Validation result: {'accuracy': 0.8413, 'f1': 0.4724, 'auc': 0.9391} | 
Early stopping counter 1 of 3



In [16]:
# load the best model
model, _, _, best_epoch = TrainUtil.load_model(model, optimizer, lr_scheduler, device, filepath=best_model_path)

train_loss, train_result, train_probs = train_util.evaluate(model, train_dataloader, best_epoch, 'Train')
# no need to reevaluate if the validation set if the last model is the best one
if best_epoch != epoch:
    val_loss, val_result, val_probs = train_util.evaluate(model, validation_dataloader, best_epoch, 'Validation')
test_loss, test_result, test_probs = train_util.evaluate(model, test_dataloader, best_epoch, 'Test')

Loaded best model from epoch 2


Epoch 2 (Train):   0%|          | 0/3609 [00:00<?, ?it/s]

Epoch 2 (Validation):   0%|          | 0/1129 [00:00<?, ?it/s]

Epoch 2 (Test):   0%|          | 0/609 [00:00<?, ?it/s]

## Dump results and others

In [17]:
with open(root + 'train.pkl', 'rb') as input_file:
    train_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'validation.pkl', 'rb') as input_file:
    validation_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(root + 'test.pkl', 'rb') as input_file:
    test_tokenized = pickle.load(input_file)
    input_file.close()

In [18]:
train_util.dump_results(
    result_dir,train_probs, train_tokenized, 
    val_probs, validation_tokenized, test_probs, test_tokenized
)

## Dump config

In [19]:
import json

config_dict = dict(Config.__dict__)
# exclude hidden variables
keys = list(config_dict.keys())
for key in keys:
    if key.startswith('__'):
        del config_dict[key]
        
with open(os.path.join(result_dir, 'config.json'), 'w') as output:
    json.dump(config_dict, output, indent=4)